In [4]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install pyarrow
!pip install sklearn
!pip install xgboost
!pip install fastparquet

In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import pyarrow as pa
import sklearn as sk
from sklearn.model_selection import KFold, cross_val_score
from sklearn import ensemble
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [6]:
df = pd.read_parquet(r"/Users/j.runge/Downloads/d14_d30_data_v3.parquet", engine= 'fastparquet')

In [7]:
df

,first_login,d1_spend,d3_spend,d7_spend,d14_spend,d30_spend,campaign,media_source,m_1,m_2,...,sum_p_13,len_p_14,count_p_15,count_p_20,sum_p_16,sum_p_17,first_p_18,avg_p_19,sum_p_19,count_p_19
0,2021-12-18,3.779171,3.779171,3.849837,3.849837,3.849837,c_0,ms_0,Motorola,Moto E6s,...,25.85,3,27,<NA>,<NA>,<NA>,<NA>,4.060000,4.06,1
1,2021-12-18,0.222576,0.536247,0.536247,0.536247,0.536247,c_1,ms_0,OPPO,CPH2127,...,NaN,<NA>,19,2,<NA>,<NA>,<NA>,2.000000,2.00,1
4,2021-06-11,0.049088,0.049088,0.049088,0.049088,0.049088,c_2,ms_1,Vivo,1907,...,NaN,<NA>,37,<NA>,<NA>,<NA>,<NA>,6.928000,34.64,5
5,2021-08-17,NaN,NaN,NaN,NaN,0.761715,c_1,ms_0,Lenovo,TB-X606V,...,NaN,<NA>,8,<NA>,<NA>,<NA>,<NA>,5.000000,5.00,1
6,2021-08-10,0.377731,0.377731,0.377731,0.377731,0.377731,c_3,ms_1,OPPO,CPH2127,...,NaN,<NA>,4,<NA>,<NA>,<NA>,<NA>,2.390000,2.39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331914,2021-10-01,0.048490,0.048490,0.048490,0.048490,0.048490,c_1,ms_0,Samsung,SM-J700F,...,NaN,<NA>,2,<NA>,<NA>,<NA>,<NA>,4.000000,4.00,1
331915,2021-12-03,0.299588,0.299588,0.299588,0.299588,0.299588,c_0,ms_0,None,None,...,NaN,<NA>,2,<NA>,<NA>,<NA>,<NA>,2.000000,2.00,1
331916,2021-12-03,NaN,NaN,79.165510,118.631176,119.051302,c_60,ms_1,Vivo,Vivo 2015,...,NaN,<NA>,85,<NA>,0,143,<NA>,25.777778,232.00,9
331917,2021-02-06,0.641836,0.641836,0.641836,0.641836,0.641836,c_5,ms_1,OPPO,CPH2015,...,NaN,<NA>,8,<NA>,<NA>,<NA>,<NA>,2.480000,2.48,1


In [8]:
df = df.fillna(0)

In [9]:
df = df.head(20000)

In [10]:
dfx = df.drop(columns = ['d30_spend','first_login','count_p_10','avg_p_13','count_p_13','sum_p_13','len_p_14','count_p_20','sum_p_16','sum_p_17','first_p_18','count_p_11','campaign','media_source','m_1','m_2','u_3','u_1','u_4','u_2','p_3'])

In [11]:
dfx

,d1_spend,d3_spend,d7_spend,d14_spend,m_3,m_4,avg_p_1,count_p_1,max_p_2,p_4,...,sum_p_8,count_p_8,avg_p_9,sum_p_9,count_p_9,count_p_12,count_p_15,avg_p_19,sum_p_19,count_p_19
0,3.779171,3.779171,3.849837,3.849837,0,0,40.583161,72,0.01,0.500000,...,7.350000,72,0.072639,5.230003,72,32,27,4.060,4.06,1
1,0.222576,0.536247,0.536247,0.536247,0,0,64.728265,40,0.01,0.500000,...,2.000000,40,0.012500,0.500000,40,1,19,2.000,2.00,1
4,0.049088,0.049088,0.049088,0.049088,0,0,91.734774,3,0.01,0.000000,...,0.150000,3,0.000000,0.000000,3,5,37,6.928,34.64,5
5,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0,0.00,0.000000,...,0.000000,0,0.000000,0.000000,0,4,8,5.000,5.00,1
6,0.377731,0.377731,0.377731,0.377731,0,0,62.417732,18,0.01,0.500000,...,0.900000,18,0.025000,0.450000,18,2,4,2.390,2.39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22696,0.266826,0.266826,0.266826,0.266826,0,0,43.615445,18,0.01,0.666667,...,0.900000,18,0.016111,0.290000,18,2,12,2.590,2.59,1
22697,0.129286,0.129286,0.129286,0.129286,0,0,44.274785,8,0.01,0.500000,...,0.400000,8,0.006250,0.050000,8,5,12,2.000,2.00,1
22698,0.225761,0.225761,0.225761,0.225761,0,0,27.636236,11,0.01,0.500000,...,0.550000,11,0.031818,0.350000,11,18,23,12.750,102.00,8
22699,44.236229,116.113346,264.589331,267.635450,0,0,66.920192,1253,0.01,0.666667,...,851.000001,1253,0.212522,266.290000,1253,73,236,32.809,656.18,20


## Regression

In [12]:
y_train = df.d30_spend
x_train = dfx

In [13]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
for train_index, test_index in kf.split(x_train, y_train):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 16000, Test set:4000
Fold:2, Train set: 16000, Test set:4000
Fold:3, Train set: 16000, Test set:4000
Fold:4, Train set: 16000, Test set:4000
Fold:5, Train set: 16000, Test set:4000


In [14]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

In [15]:
score = cross_val_score(ensemble.RandomForestRegressor(random_state= 42), x_train, y_train,
                        cv=kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [ -545187.24455126 -1404138.67374892  -545349.60232586  -357590.17359122
  -357830.18284882]
rmse= 801.26


## Classification

In [16]:
df.d30_spend = df.d30_spend.apply(lambda x:1 if x > 0 else 0)

In [17]:
y_train_clf = df.d30_spend

In [18]:
score = cross_val_score(ensemble.RandomForestClassifier(random_state= 42), x_train, y_train_clf,
                        cv=kf, scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-0.      -0.      -0.0005  -0.00025 -0.     ]
rmse= 0.01


Classification XGboost


In [19]:
XGB = XGBClassifier() 

In [20]:
score=cross_val_score(XGB,x_train,y_train_clf,cv=kf,scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [-0.00025 -0.      -0.00075 -0.      -0.     ]
rmse= 0.01


Regression

In [21]:
XGB = XGBRegressor()

In [22]:
score=cross_val_score(XGB,x_train,y_train,cv=kf,scoring="neg_mean_squared_error")
print(f'Scores for each fold: {score}')
rmse(score.mean())

Scores for each fold: [ -642841.96615471 -1146071.75769506  -565867.5950321   -323401.57366466
  -938318.95049786]
rmse= 850.47
